In [1]:
%matplotlib inline
import numpy as np

> **作者**:	Valentin Haenel

本章包含了许多可以在Python使用原生代码（主要是C/C++）方式的*介绍*，这个过程通常被称为*封装*。本章的目的是给你有哪些技术存在已经各自有优劣式的一点儿感觉，这样你可以根据你的具体需求选择适合的方式。无论如何，只要你开始做封装，你几乎都必然需要咨询你选定技术的文档。

---
章节内容

    简介
    Python-C-Api
    Ctypes
    SWIG
    Cython
    总结
    进一步阅读和参考
    练习

---

## 2.8.1 简介

本章将涵盖一下技术：

- Python-C-Api
- Ctypes
- SWIG (简化封装器和接口生成器)
- Cython

These four techniques are perhaps the most well known ones, of which Cython is probably the most advanced one and the one you should consider using first. The others are also important, if you want to understand the wrapping problem from different angles. Having said that, there are other alternatives out there, but having understood the basics of the ones above, you will be in a position to evaluate the technique of your choice to see if it fits your needs.